In [0]:
import sys
sys.path.append("/Workspace/Users/kgenuins@emeal.nttdata.com/project-insight-lab-databricks")

from Config.spark_config import apply_storage_config
from Config.storage_config import *
from Utils.utils import *

from pyspark.sql.functions import (
    col,                    
    when,                   
    lit,                    
    trim,                   
    upper,                 
    lower,                         
    coalesce,              
    concat,                            
    substring,             
    length,                
    cast,                  
    round,                
    sum,                   
    count,                 
    avg,                   
    max,                   
    min,                   
    countDistinct,         
    current_timestamp,     
    year,                  
    month,                 
    datediff,              
    to_date,               
    date_format,           
    rank,                  
    dense_rank,            
    lag,                  
    lead,
    decode,
    encode ,
    regexp_replace,
    lpad, 
    translate,
    collect_list,
    explode,
    split,
    sha2,
    concat_ws             
)

from pyspark.sql.types import (
    StringType,
    IntegerType,
    DoubleType,
    DateType,
    TimestampType,
    DecimalType
)

from pyspark.sql import Window, DataFrame

apply_storage_config(spark)

In [0]:
dbutils.fs.ls(bronze_path)

Definição dos paths

In [0]:
path_storage_bronze = f"{bronze_path}"
path_storage_silver = f"{silver_path}"


## Tratamentos e filtros

#### Tratamentos NCM

In [0]:
# NCM - Tratamento e filtros

# Abrindo o arquivo do NCM na Bronze
df_ncm = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/ncm/")

# Recuperando as colunas
columns = df_ncm.columns

# -- Sequência de tratamentos --

# 1. UTF-8
#df_ncm = latin1_to_utf8(df_ncm, ["NO_NCM_POR"])

# 2. strip/trim
df_ncm = strip_df(df_ncm, ["NO_NCM_POR"])

# 3. Colunas em branco ou somente com espaços terão valor nulificado
df_ncm = normalize_blanks_to_null(df_ncm, ["NO_NCM_POR"])

# 4. LPAD dos NCM
df_ncm = df_ncm.withColumn("CO_NCM", lpad(col('CO_NCM'), 8, "0") )
df_ncm = df_ncm.withColumn("CO_SH6", lpad(col('CO_SH6'), 6, "0") )

display(df_ncm)

In [0]:
display(df_ncm)

In [0]:
df_ncm.printSchema()

In [0]:
display(df_ncm.groupby("CO_NCM", "CO_SH6").count())

In [0]:
# Removendo as colunas NO_NCM_ESP, NO_NCM_ING
df_ncm_filtrado = df_ncm.drop("NO_NCM_ESP", "NO_NCM_ING", 'ingestion_dt','tipo_operacao','origin_path_name')
display(df_ncm_filtrado)

In [0]:
# Salvando no storage da silver
(
    df_ncm_filtrado.write
    .mode("overwrite")
    .format("delta")
    .save(f"{path_storage_silver}/balancacomercial/ncm_tratado")
)

In [0]:
df_ncm_filtrado.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("silver.ncm_tratado")

#### Tratamentos NCM SH

In [0]:
df_ncm_sh = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_sh")

In [0]:
display(df_ncm_sh)

In [0]:
# ncm_sh - Tratamento e filtros

# Abrindo o arquivo do NCM_SH na Bronze
df_ncm_sh = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_sh")

# Recuperando as colunas
columns = df_ncm_sh.columns

# -- Sequência de tratamentos --

# 1. UTF-8
#df_ncm_sh = latin1_to_utf8(df_ncm_sh, ["NO_ncm_sh_POR"])

# 2. strip/trim
df_ncm_sh = strip_df(df_ncm_sh, ["NO_SH6_POR", "NO_SH4_POR", "NO_SH2_POR", "NO_SEC_POR"])

# 3. Colunas em branco ou somente com espaços terão valor nulificado
df_ncm_sh = normalize_blanks_to_null(df_ncm_sh, ["NO_SH6_POR", "NO_SH4_POR", "NO_SH2_POR", "NO_SEC_POR"])

# 4. LPAD dos ncm_sh
df_ncm_sh = df_ncm_sh.withColumn("CO_SH6", lpad(col('CO_SH6'), 6, "0") )
df_ncm_sh = df_ncm_sh.withColumn("CO_SH4", lpad(col('CO_SH4'), 4, "0") )
df_ncm_sh = df_ncm_sh.withColumn("CO_SH2", lpad(col('CO_SH2'), 2, "0") )


In [0]:
df_ncm_sh_filtrado = df_ncm_sh.drop('NO_SH6_ESP','NO_SH6_ING','NO_SH4_ESP','NO_SH4_ING','NO_SH2_ESP','NO_SH2_ING','NO_SEC_ESP','NO_SEC_ING','ingestion_dt','tipo_operacao','origin_path_name')

display(df_ncm_sh_filtrado)


In [0]:
# Salvando no storage da silver
(
    df_ncm_sh_filtrado.write
    .mode("overwrite")
    .format("delta")
    .save(f"{path_storage_silver}/balancacomercial/ncm_sh_tratado")
)

In [0]:
df_ncm_sh_filtrado.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("silver.ncm_sh_tratado")

#### Tratamentos NCM ISIC

In [0]:
df_ncm_isic = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_isic/")

In [0]:
display(df_ncm_isic)

In [0]:
df_ncm_isic.columns

In [0]:
# ncm_isic - Tratamento e filtros

# Abrindo o arquivo do ncm_isic na Bronze
df_ncm_isic = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_isic/")

# Recuperando as colunas
columns = df_ncm_isic.columns

# -- Sequência de tratamentos --

# 1. UTF-8
#df_ncm_isic = latin1_to_utf8(df_ncm_isic, ["NO_ncm_isic_POR"])

# 2. strip/trim
df_ncm_isic = strip_df(df_ncm_isic, ["NO_ISIC_CLASSE", "NO_ISIC_GRUPO", "NO_ISIC_DIVISAO", "NO_ISIC_SECAO"])

# 3. Colunas em branco ou somente com espaços terão valor nulificado
df_ncm_isic = normalize_blanks_to_null(df_ncm_isic, ["NO_ISIC_CLASSE", "NO_ISIC_GRUPO", "NO_ISIC_DIVISAO", "NO_ISIC_SECAO"])

# 4. LPAD dos ncm_isic
df_ncm_isic = df_ncm_isic.withColumn("CO_ISIC_CLASSE", lpad(col('CO_ISIC_CLASSE'), 4, "0") )
df_ncm_isic = df_ncm_isic.withColumn("CO_ISIC_GRUPO", lpad(col('CO_ISIC_GRUPO'), 3, "0") )
df_ncm_isic = df_ncm_isic.withColumn("CO_ISIC_DIVISAO", lpad(col('CO_ISIC_DIVISAO'), 2, "0") )


display(df_ncm_isic)

In [0]:

df_ncm_isic_filtrado = df_ncm_isic.drop('NO_ISIC_CLASSE_ESP','NO_ISIC_CLASSE_ING','NO_ISIC_GRUPO_ESP','NO_ISIC_GRUPO_ING','NO_ISIC_DIVISAO_ESP','NO_ISIC_DIVISAO_ING','NO_ISIC_SECAO_ESP','NO_ISIC_SECAO_ING','ingestion_dt','tipo_operacao','origin_path_name')

display(df_ncm_isic_filtrado)

In [0]:
# Salvando no storage da silver
(
    df_ncm_isic_filtrado.write
    .mode("overwrite")
    .format("delta")
    .save(f"{path_storage_silver}/balancacomercial/ncm_isic_tratado")
)

In [0]:
df_ncm_isic_filtrado.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("silver.ncm_isic_tratado")

##### Tratamentos NCM_CGCE

In [0]:
df_ncm_cgce = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_cgce/")

In [0]:
display(df_ncm_cgce)

In [0]:
df_ncm_cgce.columns

In [0]:
# ncm_cgce - Tratamento e filtros

# Abrindo o arquivo do ncm_cgce na Bronze
df_ncm_cgce = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_cgce/")

# Recuperando as colunas
columns = df_ncm_cgce.columns

# -- Sequência de tratamentos --

# 1. UTF-8
#df_ncm_cgce = latin1_to_utf8(df_ncm_cgce, ["NO_ncm_cgce_POR"])

# 2. strip/trim
df_ncm_cgce = strip_df(df_ncm_cgce, ["NO_CGCE_N3", "NO_CGCE_N2", "NO_CGCE_N1"])

# 3. Colunas em branco ou somente com espaços terão valor nulificado
df_ncm_cgce = normalize_blanks_to_null(df_ncm_cgce, ["NO_CGCE_N3", "NO_CGCE_N2", "NO_CGCE_N1"])

# 4. LPAD dos ncm_cgce
df_ncm_cgce = df_ncm_cgce.withColumn("CO_CGCE_N3", lpad(col('CO_CGCE_N3'), 3, "0") )
df_ncm_cgce = df_ncm_cgce.withColumn("CO_CGCE_N2", lpad(col("CO_CGCE_N2"), 2, "0") )
df_ncm_cgce = df_ncm_cgce.withColumn("CO_CGCE_N1", lpad(col('CO_CGCE_N1'), 1, "0") )


display(df_ncm_cgce)

In [0]:

# df_ncm_cgce_filtrado
df_ncm_cgce_filtrado = df_ncm_cgce.drop('NO_CGCE_N3_ESP','NO_CGCE_N3_ING','NO_CGCE_N2_ESP','NO_CGCE_N2_ING','NO_CGCE_N1_ESP','NO_CGCE_N1_ING','ingestion_dt','tipo_operacao','origin_path_name')
display(df_ncm_cgce_filtrado)


In [0]:
# Salvando no storage da silver
(
    df_ncm_cgce_filtrado.write
    .mode("overwrite")
    .format("delta")
    .save(f"{path_storage_silver}/balancacomercial/ncm_cgce_tratado")
)

In [0]:
df_ncm_cgce_filtrado.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("silver.ncm_cgce_tratado")

#### Tratamentos NCM_FAT_AGREG

In [0]:
# ncm_fat_agreg - Tratamento e filtros

# Abrindo o arquivo do ncm_fat_agreg na Bronze
df_ncm_fat_agreg = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_fat_agreg/")

# Recuperando as colunas
columns = df_ncm_fat_agreg.columns

# -- Sequência de tratamentos --

# 1. UTF-8
#df_ncm_fat_agreg = latin1_to_utf8(df_ncm_fat_agreg, ["NO_ncm_fat_agreg_POR"])

# 2. strip/trim
df_ncm_fat_agreg = strip_df(df_ncm_fat_agreg, ["NO_FAT_AGREG", "NO_FAT_AGREG_GP"])

# 3. Colunas em branco ou somente com espaços terão valor nulificado
df_ncm_fat_agreg = normalize_blanks_to_null(df_ncm_fat_agreg, ["NO_FAT_AGREG", "NO_FAT_AGREG_GP"])

# 4. LPAD dos ncm_fat_agreg
df_ncm_fat_agreg = df_ncm_fat_agreg.withColumn("CO_FAT_AGREG", lpad(col('CO_FAT_AGREG'), 2, "0") )

# 5. Limpeza de acentos
df_ncm_fat_agreg = remove_accents(df_ncm_fat_agreg, ["NO_FAT_AGREG", "NO_FAT_AGREG_GP"])


display(df_ncm_fat_agreg)

In [0]:

# df_ncm_fat_agreg_filtrado
df_ncm_fat_agreg_filtrado = df_ncm_fat_agreg.drop('NO_FAT_AGREG_GP','ingestion_dt','tipo_operacao','origin_path_name')
display(df_ncm_fat_agreg_filtrado)


In [0]:
# Salvando no storage da silver
(
    df_ncm_fat_agreg_filtrado.write
    .mode("overwrite")
    .format("delta")
    .save(f"{path_storage_silver}/balancacomercial/ncm_fat_agreg_tratado")
)

In [0]:
df_ncm_fat_agreg_filtrado.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("silver.ncm_fat_agreg_tratado")

##### NBM_NCM (Não utilizada na Gold)

Ler Dados de Exportacao

### Ler Tabelas para (Dimensao produto)

- Tabelas de referencia 
-  NCM + NCM_SH + NCM_ISIC + NCM_CGCE + NCM_FAT_AGREG
 

In [0]:
# Ler tabela NCM (nomenclatura de produtos)
df_ncm = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/ncm/")
print("\nTabela NCM carregada:")

# Ler tabela ncm_cgce 
df_ncm_cgce = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_cgce")
print("\nTabela ncm_cgce carregada:")

# Ler tabela ncm_fat_agreg 
df_ncm_fat_agreg = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_fat_agreg")
print("\nTabela ncm_fat_agreg carregada:")

# Ler tabela ncm_sh 
df_ncm_sh = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_sh")
print("\nTabela ncm_sh carregada:")

# Ler tabela ncm_isic 
df_ncm_isic = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/ncm_isic")
print("\nTabela ncm_sh carregada:")



### Limpeza e Validacao de Dados

- Remover colunas não utilizavéis
- Padronizar formatos

NCM + NCM_SH + NCM_ISIC + NCM_CGCE + NCM_FAT_AGREG 

Dataframes filtrados 
- df_ncm_filtrado
- df_ncm_sh_filtrado
- df_ncm_isic_filtrado
- df_ncm_cgce_filtrado
- df_ncm_fat_agreg_filtrado



In [0]:
df_dim_produto = df_ncm_filtrado

In [0]:
df_dim_produto = df_dim_produto.join(df_ncm_sh_filtrado, on='CO_SH6', how='left') \
                    .join(df_ncm_isic_filtrado, on='CO_ISIC_CLASSE', how='left') \
                    .join(df_ncm_cgce_filtrado, on='CO_CGCE_N3', how='left') \
                    .join(df_ncm_fat_agreg_filtrado, on='CO_FAT_AGREG', how='left')



In [0]:
dim_produto = (
    df_dim_produto
    .withColumn(
        "sk_produto",
        sha2(concat_ws("||", col("CO_NCM")), 256)
    )
)

In [0]:
display(dim_produto)

CRIAR DIMENSÃO PRODUTO

In [0]:
# Salvar na Silver
dim_produto.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "True") \
    .save(f"{silver_path}dim_produto/")
    

In [0]:
spark.sql("""
          CREATE SCHEMA IF NOT EXISTS gold;
          """)

In [0]:
dim_produto.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("gold.dim_produtos")

In [0]:
%sql
SELECT * FROM gold.dim_produtos;

DIMENSÃO - DIM_LOCALIDADE_BR (UF + UF_MUN)

In [0]:
# Ler tabela uf
df_uf = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/uf")
print("\nTabela uf carregada:")
display(df_uf.limit(20))

# Ler tabela uf_mun 
df_uf_mun = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/uf_mun")
print("\nTabela uf_mun carregada:")
display(df_uf_mun.limit(20))


In [0]:
df_uf_mun = df_uf_mun.withColumn("SG_UF", col("SG_UF").cast("string"))
df_uf = df_uf.withColumn("SG_UF", col("SG_UF").cast("string"))


In [0]:
df_uf_mun = df_uf_mun.withColumn("SG_UF", upper(trim(col("SG_UF"))))
df_uf = df_uf.withColumn("SG_UF", upper(trim(col("SG_UF"))))

In [0]:
display(df_uf_mun)

In [0]:
display(df_uf)

In [0]:
display(df_uf_mun.groupby("NO_MUN", "NO_MUN_MIN").count())

In [0]:
df_uf_filtrado = df_uf.drop('ingestion_dt','tipo_operacao','origin_path_name')
display(df_uf_filtrado)

In [0]:
df_uf_mun_filtrado = df_uf_mun.drop('ingestion_dt','tipo_operacao','origin_path_name', 'NO_MUN')
display(df_uf_filtrado)

In [0]:

df_dim_localidade_br = df_uf_mun_filtrado.join(
        df_uf_filtrado,
        df_uf_mun_filtrado.SG_UF == df_uf_filtrado.SG_UF,
        "left"
    ).select(
        df_uf_filtrado.CO_UF,
        df_uf_mun_filtrado.CO_MUN_GEO,
        df_uf_mun_filtrado.NO_MUN_MIN,
        df_uf_mun_filtrado.SG_UF,
        df_uf_filtrado.NO_UF,
        df_uf_filtrado.NO_REGIAO
    )

In [0]:
display(df_dim_localidade_br)

In [0]:
dim_localidade_br = df_dim_localidade_br.withColumn(
    "sk_localidade",
    sha2(concat_ws("||", col("NO_UF"), col("CO_MUN_GEO")), 256)
)

In [0]:
display(dim_localidade_br)

In [0]:
display(dim_localidade_br.groupby("SG_UF", "NO_REGIAO").count())

In [0]:
# Salvar na Silver
dim_localidade_br.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "True") \
    .save(f"{silver_path}dim_localidade_br/")
    

In [0]:
dim_localidade_br.write \
  .format("delta") \
  .mode("overwrite") \
  .saveAsTable("gold.dim_localidades_br")

In [0]:
%sql

SELECT * FROM gold.dim_localidades_br

### 4.1. Transformacao de Tipos de Dados

- Dados brutos vem como STRING
- Para fazer calculos, precisamos de INT, DOUBLE, etc
- Economiza espaco de armazenamento
- Melhora performance de queries

# DIM_GEOGRAFIA: Junte PAIS + PAIS_BLOCO.



In [0]:
# Ler tabela pais
df_pais = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/pais")
print("\nTabela uf carregada:")
display(df_pais.limit(10))

In [0]:
# Ler tabela pais_bloco
df_pais_bloco = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/tb_aux/pais_bloco")
print("\nTabela uf carregada:")
display(df_pais_bloco.limit(10))

In [0]:
# Removendo as colunas _ESP, _ING
df_pais_filtrado = df_pais.drop("NO_PAIS_ESP", "NO_PAIS_ING", 'ingestion_dt','tipo_operacao','origin_path_name')
display(df_pais_filtrado.limit(10))

In [0]:
# Removendo as colunas _ESP, _ING
df_pais_bloco_filtrado = df_pais_bloco.drop("NO_BLOCO_ESP", "NO_BLOCO_ING", 'ingestion_dt','tipo_operacao','origin_path_name')
display(df_pais_bloco_filtrado.limit(10))

In [0]:
# Verificar valores nulos em cada coluna
total_linhas = df_pais_filtrado.count()

for coluna in df_pais_filtrado.columns:
    nulos_na_coluna = total_linhas - df_pais_filtrado.filter(df_pais_filtrado[coluna].isNull()).count()
    print(f"{coluna}: {nulos_na_coluna} nulos")


In [0]:
# Verificar valores nulos em cada coluna
total_linhas = df_pais_bloco_filtrado.count()

for coluna in df_pais_bloco_filtrado.columns:
    nulos_na_coluna = total_linhas - df_pais_bloco_filtrado.filter(df_pais_bloco_filtrado[coluna].isNull()).count()
    print(f"{coluna}: {nulos_na_coluna} nulos")

In [0]:
df_dim_geografia = df_pais_filtrado

In [0]:
df_dim_geografia = df_pais_filtrado.join(df_pais_bloco_filtrado, on='CO_PAIS', how='left') 
display(df_dim_geografia)
                    

In [0]:
df_dim_geografia.count()

In [0]:
# alterar valores nulos por 'Não Definido'
df_dim_geografia = df_dim_geografia.fillna('Não Definido', subset=['NO_BLOCO'])
df_dim_geografia = df_dim_geografia.fillna('Não Definido', subset=['CO_BLOCO'])

df_dim_geografia.display()

In [0]:
dim_geografia = (
    df_dim_geografia
    .withColumn(
        "sk_geografia",
        sha2(concat_ws("||", col("CO_PAIS")), 256)
    )
)

In [0]:
display(dim_geografia)

In [0]:
# Salvar na Silver
dim_geografia.write \
 \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "True") \
    .save(f"{silver_path}dim_geografia/")

In [0]:
# gravar tabela na camada Silver
dim_geografia.write \
    .mode("overwrite") \
    .saveAsTable("gold.dim_geografia")

In [0]:
%sql
select * from gold.dim_geografia limit 10;

#  Ler Tabelas para (Dimensao TEMPO)

- Tabelas de referencia 
- CO_ANO
- CO_ANO

In [0]:
"""df_exp = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/exp/")
print("\nTabela carregada:")

display(df_exp.limit(10))"""

In [0]:
"""df_imp = spark.read.format("delta").load(f"{path_storage_bronze}/balancacomercial/imp/")
print("\nTabela NCM carregada:")

display(df_imp.limit(10))
"""

In [0]:
df_exp_filtrado.describe()

In [0]:
#excluir colunas
"""df_exp_filtrado = df_exp.drop('ingestion_dt', 'mes', 'ano','ingestion_dt','origin_path_name')
"""

In [0]:
"""
#tranformar colunas
df_exp_filtrado = df_exp_filtrado.select(
    col("CO_ANO").cast(IntegerType()),
    col("CO_MES").cast(IntegerType()),

     # Colunas de identificacao (manter como STRING)
    col("CO_NCM").alias("CO_NCM"),
    col("CO_UNID").alias("CO_UNID"),
    col("CO_PAIS").alias("CO_PAIS"),
    col("SG_UF_NCM").alias("SG_UF_NCM"),
    col("CO_VIA").alias("CO_VIA"),
    col("CO_URF").alias("CO_URF"),

    col("QT_ESTAT").cast(IntegerType()),
    col("KG_LIQUIDO").cast(IntegerType()),
    col("VL_FOB").cast(IntegerType()),
    col("tipo_operacao").alias('TIPO_OPERACAO'),
    
    # Metadados
    current_timestamp().alias("DATA_TRANSFORMACAO")
)
display(df_exp_filtrado)

"""

In [0]:

"""df_exp = spark.read.parquet(f"{path_silver}/exportacao").select("CO_ANO", "CO_MES")
df_imp = spark.read.parquet(f"{path_silver}/importacao").select("CO_ANO", "CO_MES")

df_datas_base = df_exp.union(df_imp).distinct()

dim_tempo = df_datas_base.withColumn("CO_TRIMESTRE", F.ceil(F.col("CO_MES") / 3)) \
                         .withColumn("CO_SEMESTRE", F.ceil(F.col("CO_MES") / 6)) \
                         .withColumn("NO_TRIMESTRE", F.concat(F.col("CO_TRIMESTRE"), F.lit("º Trimestre"))) \
                         .withColumn("NO_SEMESTRE", F.concat(F.col("CO_SEMESTRE"), F.lit("º Semestre"))) \
                         .orderBy("CO_ANO", "CO_MES")
"""



In [0]:
# unir as 2 tabelas
df_tempo_distinc = df_tempo_exp.union(df_tempo_imp).distinct()

%md
NCM + NCM_SH + NCM_ISIC + NCM_CGCE + NCM_FAT_AGREG 

Dataframes filtrados 
- df_ncm_filtrado
- df_ncm_sh_filtrado
- df_ncm_isic_filtrado
- df_ncm_cgce_filtrado
- df_ncm_fat_agreg_filtrado
- df_exp_filtrado
- df_exp_filtrado